<a href="https://colab.research.google.com/github/sahana-github/Machine-learning-Demo/blob/main/HeartDiseasesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Dependencies


In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

Data Collection and Processing

In [23]:
#load csv file to pandas dataframe
heart_data=pd.read_csv("/content/heart_disease_uci.csv")

In [24]:
#print first 5 rows of the dataset
heart_data.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [25]:
heart_data.tail()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
915,916,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,917,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,919,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0
919,920,62,Male,VA Long Beach,atypical angina,120.0,254.0,False,lv hypertrophy,93.0,True,0.0,NaN,NaN,NaN,1


In [26]:
#number of rows and columns in dataset
heart_data.shape

(920, 16)

In [27]:
#getting some info about the data
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


In [28]:
#convert object to numeric
heart_data1 = heart_data.copy().apply(pd.to_numeric, errors='coerce')


In [40]:
heart_data1 = (heart_data.copy()
                .dropna(subset=heart_data.select_dtypes(include=['object']).columns)
                .apply(pd.to_numeric, errors='coerce'))


In [29]:
#checking for missing values
heart_data.isnull().sum()

id            0
age           0
sex           0
dataset       0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64

In [30]:
#Statistical measure of data
heart_data.describe()

,id,age,trestbps,chol,thalch,oldpeak,ca,num
count,920.000000,920.000000,861.000000,890.000000,865.000000,858.000000,309.000000,920.000000
mean,460.500000,53.510870,132.132404,199.130337,137.545665,0.878788,0.676375,0.995652
std,265.725422,9.424685,19.066070,110.780810,25.926276,1.091226,0.935653,1.142693
min,1.000000,28.000000,0.000000,0.000000,60.000000,-2.600000,0.000000,0.000000
25%,230.750000,47.000000,120.000000,175.000000,120.000000,0.000000,0.000000,0.000000
50%,460.500000,54.000000,130.000000,223.000000,140.000000,0.500000,0.000000,1.000000
75%,690.250000,60.000000,140.000000,268.000000,157.000000,1.500000,1.000000,2.000000
max,920.000000,77.000000,200.000000,603.000000,202.000000,6.200000,3.000000,4.000000


In [31]:
#checking distribution of target variable
heart_data['num'].value_counts()

num
0    411
1    265
2    109
3    107
4     28
Name: count, dtype: int64

splitting features and target


In [41]:
X=heart_data1.drop(columns='num',axis=1)
Y=heart_data1['num']

In [33]:
print(X)

      id  age  sex  dataset  cp  trestbps   chol  fbs  restecg  thalch  exang  \
0      1   63  NaN      NaN NaN     145.0  233.0  1.0      NaN   150.0    0.0   
1      2   67  NaN      NaN NaN     160.0  286.0  0.0      NaN   108.0    1.0   
2      3   67  NaN      NaN NaN     120.0  229.0  0.0      NaN   129.0    1.0   
3      4   37  NaN      NaN NaN     130.0  250.0  0.0      NaN   187.0    0.0   
4      5   41  NaN      NaN NaN     130.0  204.0  0.0      NaN   172.0    0.0   
..   ...  ...  ...      ...  ..       ...    ...  ...      ...     ...    ...   
915  916   54  NaN      NaN NaN     127.0  333.0  1.0      NaN   154.0    0.0   
916  917   62  NaN      NaN NaN       NaN  139.0  0.0      NaN     NaN    NaN   
917  918   55  NaN      NaN NaN     122.0  223.0  1.0      NaN   100.0    0.0   
918  919   58  NaN      NaN NaN       NaN  385.0  1.0      NaN     NaN    NaN   
919  920   62  NaN      NaN NaN     120.0  254.0  0.0      NaN    93.0    1.0   

     oldpeak  slope   ca  t

splitting data into training and test data

In [42]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [43]:
print(X.shape,X_train.shape,X_test.shape)

(372, 15) (297, 15) (75, 15)


In [44]:
print(Y.shape,Y_train.shape,Y_test.shape)

(372,) (297,) (75,)


Model Training

In [45]:
model=LogisticRegression()

In [47]:
#train the LogisticRegression model with training data
try:
  # Attempt training with potential NaN/inf removal using boolean indexing
  model.fit(X_train[~np.isnan(X_train).any(axis=1) & ~np.isinf(X_train).any(axis=1)], Y_train)
except ValueError:
  # If error persists, consider alternative handling (e.g., imputation, scaling)
  pass


Model evaluation

In [50]:
try:
  # Train-predict-evaluate block
  X_train_prediction = model.predict(X_train)
  training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
  print("Training Accuracy:", training_data_accuracy)
except ValueError:
  print("Error: Training failed due to data issues. Consider data cleaning or imputation.")


Error: Training failed due to data issues. Consider data cleaning or imputation.


In [52]:
try:
  # Data cleaning and imputation
  missing_values_count = X_train.isnull().sum()

  # Decide on imputation strategy (adjust as needed)
  # Option 1: Remove rows with few missing values
  if missing_values_count.sum() / X_train.shape[0] < 0.1:  # Less than 10% missing values overall
    X_train_cleaned = X_train.dropna()
  else:
    # Option 2: Impute missing values with mean (for numerical data)
    imputer = SimpleImputer(strategy='mean')
    X_train_cleaned = imputer.fit_transform(X_train)

  # Split data into training and testing sets (e.g., 80%/20% split)
  X_train, X_test, Y_train, Y_test = train_test_split(X_train_cleaned, Y_train, test_size=0.2)

  # Train the model
  model = LogisticRegression()
  model.fit(X_train, Y_train)

  # Make predictions on the training data
  X_train_prediction = model.predict(X_train)

  # Calculate training accuracy
  training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
  print("Training Accuracy:", training_data_accuracy)

  # Make predictions on the testing data (optional)
  X_test_prediction = model.predict(X_test)

  # Calculate testing accuracy (optional)
  testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)
  print("Testing Accuracy:", testing_data_accuracy)

except ValueError:
  print("Error: Data cleaning or model training failed. Consider alternative strategies.")

Training Accuracy: 0.5358649789029536
Testing Accuracy: 0.5333333333333333


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
